In [ ]:
!pip install ultralytics -q
!pip install roboflow -q

In [ ]:
from ultralytics.nn.modules import (
    AIFI,
    C1,
    C2,
    C2PSA,
    C3,
    C3TR,
    ELAN1,
    OBB,
    PSA,
    SPP,
    SPPELAN,
    SPPF,
    A2C2f,
    AConv,
    ADown,
    Bottleneck,
    BottleneckCSP,
    C2f,
    C2fAttn,
    C2fCIB,
    C2fPSA,
    C3Ghost,
    C3k2,
    C3x,
    CBFuse,
    CBLinear,
    Classify,
    Concat,
    Conv,
    Conv2,
    ConvTranspose,
    Detect,
    DWConv,
    DWConvTranspose2d,
    Focus,
    GhostBottleneck,
    GhostConv,
    HGBlock,
    HGStem,
    ImagePoolingAttn,
    Index,
    LRPCHead,
    Pose,
    RepC3,
    RepConv,
    RepNCSPELAN4,
    RepVGGDW,
    ResNetLayer,
    RTDETRDecoder,
    SCDown,
    Segment,
    TorchVision,
    WorldDetect,
    YOLOEDetect,
    YOLOESegment,
    v10Detect,
)

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('../../myconfig.config')
print(config['roboflow']['roboflow_api_key'])


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key='u8Ur5ywKLswqwt0uMTXl')
project = rf.workspace("poacher-ifyuf").project("poacher")
version = project.version(2)
dataset = version.download("yolov11", location="../../Dataset/Test_dataset")

In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebooks/YOLO_architecture/utillity')

In [1]:
from ultralytics import nn
from ultralytics.nn import modules, tasks
import ultralytics.nn.autobackend as autobackend
from model_classes import Upsample, C2f, SPPF, DFL, Head, Bottleneck, Concat
from model_classes_v11 import C3k2, C2PSA, Conv, Attention, PSABlock, C3k, Bottleneck, C2f
from model_classes_v11_modified import CBAM, ASPP_Lite, MobileViTBlock, ConvBNAct, Channel_Attention, Spatial_Attention
from ultralytics.nn.tasks import yaml_model_load
from copy import deepcopy
import sys
import ultralytics.nn.tasks as tasks
import my_tasks
from my_tasks import my_parse_model


def change_parse_model():
    # Replace in the tasks module itself
    tasks.parse_model = my_parse_model
    # Replace in sys.modules so any "from ultralytics.nn.tasks import parse_model"
    # will now point to yours
    sys.modules['ultralytics.nn.tasks'].parse_model = my_parse_model
    return my_parse_model

def patch_ultralytics_layers(custom_classes):
    submodules = [
        nn,                          # ultralytics.nn
        modules,                     # ultralytics.nn.modules
        tasks,                       # ultralytics.nn.tasks
        autobackend,                 # ultralytics.nn.autobackend
        my_tasks
    ]

    for cls in custom_classes:
        for mod in submodules:
            mod.__dict__[cls.__name__] = cls

def build_model(model_yaml, version='n', ch=3, verbose=False, custom_classes=[]):
  parse_model = change_parse_model()
  patch_ultralytics_layers(custom_classes)
  mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)
  mod_ya['scale'] = version.lower()
  model, save  = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)

  return model,save


In [5]:
from ultralytics import YOLO

m_class = [Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat, C3k2, C2PSA, Attention, PSABlock, C3k, Bottleneck, C2f]
m_modified = [CBAM, ASPP_Lite, MobileViTBlock, ConvBNAct, Channel_Attention, Spatial_Attention]
m_class.extend(m_modified)
model_yaml = "my_yolov11l_back_modified.yaml"

model_save = build_model(model_yaml, custom_classes=m_class, version='l', verbose=False)

model = YOLO(model_yaml, verbose=True)
print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  model_classes_v11.Conv                       [3, 64, 3, 2]                 
  1                  -1  1     73984  model_classes_v11.Conv                       [64, 128, 3, 2]               
  2                  -1  2    173824  model_classes_v11.C3k2                       [128, 256, 2, True, 0.25]     
  3                  -1  1     65890  model_classes_v11_modified.CBAM              [256]                         
  4                  -1  1    590336  model_classes_v11.Conv                       [256, 256, 3, 2]              
  5                  -1  2    691712  model_classes_v11.C3k2                       [256, 512, 2, True, 0.25]     
  6                  -1  1   3380736  model_classes_v11_modified.MobileViTBlock    [512, 2]                      
  7                  -1  1   2360320  model_classes_v11.Conv                       [512

In [ ]:
# from ultralytics import YOLO

# # Load the YOLOv11l model
# model = YOLO("yolo11l.yaml",verbose=True)  # Replace with the path to your model file
# print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  model_classes_v11.Conv                       [3, 64, 3, 2]                 
  1                  -1  1     73984  model_classes_v11.Conv                       [64, 128, 3, 2]               
  2                  -1  2    173824  model_classes_v11.C3k2                       [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  model_classes_v11.Conv                       [256, 256, 3, 2]              
  4                  -1  2    691712  model_classes_v11.C3k2                       [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  model_classes_v11.Conv                       [512, 512, 3, 2]              
  6                  -1  2   2234368  model_classes_v11.C3k2                       [512, 512, 2, True]           
  7                  -1  1   2360320  model_classes_v11.Conv                       [512

In [6]:
results = model.train(data=r"C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\data.yaml", epochs=1, imgsz=64)

New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.173  Python-3.11.3 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.

train: Scanning C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\train\labels.cache... 604 images, 9 backgrounds, 0 corrupt: 100%|██████████| 604/604 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.40.1 ms, read: 51.922.5 MB/s, size: 56.0 KB)



c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset\Test_dataset\valid\labels.cache... 58 images, 1 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 182 weight(decay=0.0), 198 weight(decay=0.0005), 198 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Ultralytics_work\ultralytics\runs\detect\train2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G       6.58      5.323      4.204         68         64:  18%|█▊        | 7/38 [00:24<01:46,  3.43s/it]


KeyboardInterrupt: 